In [ ]:
import numpy as np
import pandas as pd
#读取数据集
airCompany = pd.read_csv('./Q1data/air_data.csv', encoding='gb18030')
#去除两年票价收入都为空的数据（这些数据可能是有误）
price1 = airCompany['SUM_YR_1'].notnull()
price2 = airCompany['SUM_YR_2'].notnull()
priceEmpty = price1&price2
airData = airCompany.loc[priceEmpty,:]
#选取特征构建模型（数据中无关紧要的信息不少，如会员卡号、工作城市等这些不是我们这次分析的重点）
#这里选取加入会员时间（需计算）、飞行总里程、平均飞行时间间隔、平均票价收入（没有给出，需要计算）
airDataPro = airData[['FFP_DATE','LOAD_TIME','SUM_YR_1','SUM_YR_2','SEG_KM_SUM','AVG_INTERVAL']]
#计算入会时间(单位：月)
MemberAge = pd.to_datetime(airDataPro['LOAD_TIME']) - pd.to_datetime(airDataPro['FFP_DATE'])
MemberAge = MemberAge.astype('str').str.split().str[0]
MemberAge = MemberAge.astype('int')/30
#计算平均票价收入
AVG_SUM_YR = (airDataPro['SUM_YR_1'] + airDataPro['SUM_YR_2']) / 2
#合并所选取的数据特征
airDataMerge = pd.concat([MemberAge, AVG_SUM_YR, airDataPro.iloc[:,4:]], axis = 1)
print('构建的数据分析模型特征的前5行为：\n', airDataMerge.head())
#对特征数据标准化
from sklearn.preprocessing import StandardScaler
airStandardData = StandardScaler().fit_transform(airDataMerge)
np.savez('./Q1data/airStandardData.npz',airStandardData)
print('标准化后的数据分析模型的4个特征为：\n',airStandardData[:5,:])

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
#使用肘方法选取k值
d=[]
for i in range(1,11):    #k取值1~11，做kmeans聚类，看不同k值对应的簇内误差平方和
    km=KMeans(n_clusters=i,init='k-means++',n_init=10,max_iter=300,random_state=0)
    km.fit(airStandardData)
    d.append(km.inertia_)  #inertia簇内误差平方和
#肘处的K即为最佳K值,这里我选的是4
plt.plot(range(1,11),d,marker='o')
plt.xlabel('number of clusters')
plt.ylabel('distortions')
plt.show()

In [ ]:
#使用聚类算法对航空客户进行分群
from sklearn.cluster import KMeans
airStandardData = np.load('./Q1data/airStandardData.npz')['arr_0']
#聚类个数k自己选定，如果没有使用肘方法，这里可以多尝试几种情况，看看各分群的区分度明不明显，最后确定k的取值
k=4
kmeans = KMeans(n_clusters=k, random_state=200)
fitKmeans = kmeans.fit(airStandardData)
print("查看各客户群的聚类中心：")
kmeans.cluster_centers_

In [ ]:
#统计各客户群的数目
num = pd.Series(kmeans.labels_).value_counts()
print('最终每个类别的数目：\n',num)
#根据分析结果可以看出：
#第1类客户是那种比较晚加入会员，并且花费不多，乘机频率也相对较少。这属于一般客户
#第2类客户是较早加入会员，飞行里程很大、飞行频率不高且花费意愿不错，是优质客户
#第3类客户是很早加入会员，但现在乘机频率减少花费也有减少，建议回访搞清楚其中原因
#第4类客户飞行频率最高，但花费不多，可以尝试明确其需求，针对提供更多收费服务